In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingRegressor


In [3]:
import requests
import joblib
from io import BytesIO

url_data = "https://github.com/martinezb-jose-c/credit-risk-modeling/releases/download/v0.2-alpha/modeling_data.joblib"

response = requests.get(url_data)
data = joblib.load(BytesIO(response.content))
print("Datos cargados desde GitHub Release!")

Datos cargados desde GitHub Release!


In [ ]:
clf = LogisticRegressionCV(cv=5, random_state=42, solver='liblinear')
clf.fit(X_train_final, y_train)

In [ ]:
#predictions
y_pred_proba = clf.predict_proba(X_test_final)[:, 1]

# 1. ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)

# 2. Gini
gini = 2 * roc_auc - 1

# 3. Brier Score
brier = brier_score_loss(y_test, y_pred_proba)

# 4. KS Statistic
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
ks_stat = np.max(tpr - fpr)

# 5. Logarithmic loss
logarithmic_loss = log_loss(y_test, y_pred_proba)

print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Gini:    {gini:.4f}")
print(f"Brier:   {brier:.4f}")
print(f"KS:      {ks_stat:.4f}")
print(f"Log loss:{logarithmic_loss:.4f}")